In [1]:
# Reload modules automatically
# https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

In [170]:
from collections import Counter
import logging

import numpy as np
import torch
from strn_and_rbstness.data import GraphDataset, split
from strn_and_rbstness.helper.utils import accuracy
from strn_and_rbstness.models import create_model
from strn_and_rbstness.train import _train
from common import CSBM, get_sbm_model, add_adversarial_edge

logger = logging.getLogger()
logger.setLevel(logging.INFO)

## CSBM

In [165]:
n = 1000
avg_intra_degree = 1.5 * 2 # intra_edges_per_node * 2
avg_inter_degree = 0.5 * 2
csbm = get_sbm_model(n, avg_intra_degree, avg_inter_degree)
seed = 1
X, A, y = csbm.sample(n, seed)

## Model

In [166]:
model_params = dict(
    label="GCN",
    model="DenseGCN", #GCN or DenseGCN
    n_filters=64,
    dropout=0.5
)
train_params = dict(
    loss_type="CE",
    lr=1e-2,
    weight_decay=1e-3,
    patience=300,
    max_epochs=1000,
    use_selftrain = False, 
    use_advtrain = False,
)
attack = "LocalDICEUndirected"
attack_params = dict()

# Other
split_params = {
    "strategy": "normal", # or "custom"
    "p_trn": 1,
    "p_tst": 0, # "normal" uses 1 - p_trn, only for custom split strategy
    "p_selftrn": 0 # Refers to unlabeled data, which is not test data, 
                    # only for custom split strategy
}
verbosity_params = dict(
    display_steps = 1001
)   
# Device
device = 0
if not torch.cuda.is_available():
    device == "cpu", "CUDA is not availble, set device to 'cpu'"
else:
    device = torch.device(f"cuda:{device}")
    logging.info(f"Currently on gpu device {device}")
attack_params["data_device"] = device

INFO:root:Currently on gpu device cuda:0


In [167]:
# Train
torch.manual_seed(seed)
np.random.seed(seed)
split_ids = split(y, split_params, seed)
X_gpu = torch.tensor(X, dtype=torch.float32, device=device)
A_gpu = torch.tensor(A, dtype=torch.float32, device=device)
y_gpu = torch.tensor(y, device=device)
graph = GraphDataset((X_gpu, A_gpu, y_gpu), split_ids)
model_params_trn = dict(**model_params, 
                        n_features=graph.get_n_features(), 
                        n_classes=graph.get_n_classes())
model = create_model(model_params_trn).to(device)
statistics = _train(model, graph, train_params, verbosity_params, None)

INFO:root:
Epoch    0: loss_train: 0.69339, loss_val: 0.69347, acc_train: 0.48800, acc_val: 0.49000
INFO:root:
Epoch  250: loss_train: 0.49040, loss_val: 0.59152, acc_train: 0.76800, acc_val: 0.69400


In [173]:
n_iter = 1
model.eval()
c_acc_bayes = 0 # Count nodes correctly classified by bayes classifier
c_acc_bayes_deg = Counter()  # Above but for each degree
c_acc_bayes_structure = 0 # Count nodes separable by structure alone
c_acc_bayes_structure_deg = Counter() # Above but for each degree
c_acc_bayes_feature = 0 # Count nodes separable by features alone (degree dependent
                        # doesn't make sense as features independent of connections)
c_acc_bayes_not_gnn = 0 # Decisions where BC correct but GNN wrong
c_acc_bayes_not_gnn_deg = Counter() # Above but for each degree
c_acc_gnn = 0 # Count nodes correctly classified by gnn
c_acc_gnn_deg = Counter() # Above but for each degree
c_acc_gnn_not_bayes = 0 # Decisions where GNN correctly says true even though BC violated
c_acc_gnn_not_bayes_deg = Counter() # Above but for each degree
c_acc_bayes_gnn = 0 # Count nodes correctly classified by bc & gnn
c_acc_bayes_gnn_deg = Counter() # Above but for each degree
c_degree_total = Counter() # Count degrees of all generated nodes
c_degree_acc_bayes = Counter() # Count degrees where bayes classifier is correct
c_degree_acc_gnn = Counter() # Count degrees where gnn is correct
c_degree_acc_bayes_gnn = Counter() # Count degrees where bc & gnn are correct

torch.manual_seed(seed)
np.random.seed(seed)
for i in range(n_iter):
    # ToDo: Create empty X_, A_, y_ templates & always only fill last row
    X_, A_, y_ = csbm.sample_conditional(n=1, X=X, A=A, y=y)
    deg_n = np.sum(A_[:,n])
    c_degree_total[deg_n] += 1
    # Statistics Bayes Classifier
    feature_separable, _ = csbm.feature_separability(X_, y_, [n])
    structure_separable, _ = csbm.structure_separability(A_, y_, [n])
    bayes_separable, _ = csbm.likelihood_separability(X_, A_, y_, [n])
    if bayes_separable:
        c_acc_bayes += 1
        c_acc_bayes_deg[deg_n] += 1
    if structure_separable:
        c_acc_bayes_structure += 1
        c_acc_bayes_structure_deg[deg_n] += 1
    if feature_separable:
        c_acc_bayes_feature += 1
    # Calculate GNN-prediction
    X_gpu = torch.tensor(X_, dtype=torch.float32, device=device)
    A_gpu = torch.tensor(A_, dtype=torch.float32, device=device)
    y_gpu = torch.tensor(y_, device=device)
    logits = model(X_gpu, A_gpu)
    gnn_separable = round(accuracy(logits, y_gpu, n))
    # Statistics Prediction
    if gnn_separable:
        c_acc_gnn += 1
        c_acc_gnn_deg[deg_n] += 1
        if bayes_separable:
            c_acc_bayes_gnn += 1
            c_acc_bayes_gnn_deg[deg_n] += 1
        else:
            c_acc_gnn_not_bayes += 1
            c_acc_gnn_not_bayes_deg[deg_n] += 1
    elif bayes_separable:
        c_acc_bayes_not_gnn += 1
        c_acc_bayes_not_gnn_deg[deg_n] += 1
    # Investigate Robustness
    c_robustness = 0
    while bayes_separable or gnn_separable:
        j = add_adversarial_edge(n, A_, y_) #ToDo: For speed, calc pot_neighbours once!
        A_gpu[n, j] = 1
        A_gpu[j, n] = 1

        if bayes_separable:
            bayes_separable_new, _ = csbm.likelihood_separability(X_, A_, y_, [n])
            if not bayes_separable_new:
                

        break
        # Case: Bayes & GNN-Separable
        # Case: Bayes & not-GNN-separable
        # Case: GNN & not Bayes-separable

print(f"Count BC: {c_acc_bayes}; GNN: {c_acc_gnn}")
print(f"Count Structure BC: {c_acc_bayes_structure}; Feature BC: {c_acc_bayes_feature}")
print(f"Count BC and GNN: {c_acc_bayes_gnn} ")
print(f"Count BC not GNN: {c_acc_bayes_not_gnn}; "
      f"GNN not BC: {c_acc_gnn_not_bayes}")

[1 1 0 ... 0 0 1]
[  2   3   9  10  12  15  16  18  19  20  22  23  25  26  27  29  30  31
  37  38  39  46  49  50  52  53  57  59  60  63  68  69  72  73  74  75
  79  81  82  85  88  90  91  95  98 103 104 105 106 107 115 116 117 118
 119 120 127 128 131 132 134 135 136 142 144 146 147 148 152 154 155 156
 157 159 161 165 166 167 168 169 170 172 173 175 176 179 180 181 183 184
 185 186 188 189 190 191 194 195 199 200 201 202 203 207 210 211 212 215
 219 224 226 227 229 231 234 237 241 243 245 252 253 256 257 258 261 262
 263 264 265 273 274 276 277 278 280 282 284 285 286 287 290 291 292 294
 297 298 300 302 303 304 306 308 312 314 319 320 322 323 324 325 328 331
 332 340 341 342 343 344 345 346 348 349 350 351 352 357 358 359 361 363
 365 366 367 368 369 370 372 373 374 376 379 380 383 385 387 390 391 393
 394 395 396 397 400 401 406 408 410 411 413 415 417 418 419 420 422 423
 429 430 431 433 436 439 440 443 448 450 451 452 453 454 456 457 458 459
 461 462 464 467 469 471 473 474 

In [164]:
print(c_degree_total)

Counter({3: 205, 4: 183, 5: 180, 2: 149, 6: 103, 1: 69, 7: 55, 8: 28, 0: 11, 9: 11, 10: 3, 11: 2, 12: 1})


In [90]:
np.sum(A_[:,n])

0